In [1]:
import numpy as np 
import pandas as pd
from tensorflow import keras
#import matplotlib.pyplot as plt 
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import os
import shutil
from tensorflow.keras import metrics
import matplotlib
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop

from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'keras'

In [ ]:
#y
y_list = []
for image in os.listdir('Data/y_variables/Unruled'):
    file='Data/y_variables/Unruled/'+ str(image)
    y_image = np.array(Image.open(file))
    y_image = y_image.reshape(*y_image.shape,1)
    y_list.append(y_image)
                  
y = np.array(y_list)  
y.shape

In [ ]:
#X

X_list = []
for image in os.listdir('Data/X_variables/computer_generated_lines'):
    
    file = 'Data/X_variables/computer_generated_lines/'+str(image)
    X_image = np.array(Image.open(file))
    X_image = X_image.reshape(*X_image.shape,1)
    X_list.append(X_image)
    
X = np.array(X_list)
X.shape

In [ ]:

def create_model(learn_rate, kernel_size, activation, optimizer):
    
    input_img = Input(shape=(500,400,1)) 

    x = Conv2D(32, (3, 3), activation='relu')(input_img)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = Dense(128)(x) 
    x = Conv2DTranspose(32,(3,3), activation='relu')(x)
    decoded = Conv2DTranspose(1, (3, 3), activation='linear', padding='same')(x)

    autoencoder_B = Model(input_img, decoded)
    autoencoder_B.summary()
    return autoencoder_B.compile(optimizer='adam', loss='mse', metrics='accuracy')
       

model = KerasRegressor(build_fn = create_model, batch_size=1000, epochs=10)

#now write out all the parameters you want to try out for the grid search
activation = ['relu', 'tanh', 'sigmoid', 'linear', 'leakyrelu'...]
learn_rate = [0.1, 0.2, .5, ...]
init = ['unform', 'normal', 'zero', ...]
optimizer = ['SGD', 'Adam' ...]
param_grid = dict(activation=activation, learn_rate=learn_rate, init=init, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
result = grid.fit(X, y)